# cats vs dogs, Using ImageDataGenerator 

2 class (cats va dogs), 
each class 2000 image, each class 1000 image for train, 500 for validation, 500 for test.


### make dirctories:

In [104]:
import os 


original_dataset_dir='./dogs-vs-cats' 
our_dataset_dir= './database'

os.mkdir(our_dataset_dir)


train_dir=os.path.join(our_dataset_dir,'train') 
validation_dir=os.path.join(our_dataset_dir,'validation') 
test_dir=os.path.join(our_dataset_dir,'test') 

os.mkdir(train_dir) 
os.mkdir(validation_dir) 
os.mkdir(test_dir)


train_cats_dir=os.path.join(train_dir,'cats') 
train_dogs_dir=os.path.join(train_dir,'dogs') 
validation_cats_dir=os.path.join(validation_dir,'cats') 
validation_dogs_dir=os.path.join(validation_dir,'dogs') 
test_cats_dir=os.path.join(test_dir,'cats') 
test_dogs_dir=os.path.join(test_dir,'dogs') 

os.mkdir(train_cats_dir) 
os.mkdir(train_dogs_dir)
os.mkdir(validation_cats_dir)
os.mkdir(validation_dogs_dir)
os.mkdir(test_cats_dir)
os.mkdir(test_dogs_dir)

### fill dirctories:

In [105]:
import  shutil 


fnames_cat_train=['cat.{}.jpg'.format(i) for i in range(1000)] 
for fname in fnames_cat_train:
    src=os.path.join(original_dataset_dir, fname)
    dst=os.path.join(train_cats_dir,fname)
    shutil.copyfile(src,dst) 
    

fnames_cat_validation=['cat.{}.jpg'.format(i) for i in range(1000, 1500)] 
for fname in fnames_cat_validation:
    src=os.path.join(original_dataset_dir, fname)
    dst=os.path.join(validation_cats_dir,fname)
    shutil.copyfile(src,dst)
    
    
fnames_cat_test=['cat.{}.jpg'.format(i) for i in range(1500, 2000)] 
for fname in fnames_cat_test:
    src=os.path.join(original_dataset_dir, fname)
    dst=os.path.join(test_cats_dir,fname)
    shutil.copyfile(src,dst)
    
####################dog s part:

fnames_dog_train=['dog.{}.jpg'.format(i) for i in range(1000)] 
for fname in fnames_dog_train:
    src=os.path.join(original_dataset_dir, fname)
    dst=os.path.join(train_dogs_dir,fname)
    shutil.copyfile(src,dst) 
    

fnames_dog_validation=['dog.{}.jpg'.format(i) for i in range(1000, 1500)] 
for fname in fnames_dog_validation:
    src=os.path.join(original_dataset_dir, fname)
    dst=os.path.join(validation_dogs_dir,fname)
    shutil.copyfile(src,dst)
    
    
fnames_dog_test=['dog.{}.jpg'.format(i) for i in range(1500, 2000)] 
for fname in fnames_dog_test:
    src=os.path.join(original_dataset_dir, fname)
    dst=os.path.join(test_dogs_dir,fname)
    shutil.copyfile(src,dst)

###  sanity check

In [107]:
print('total training cat image:', len(os.listdir(train_cats_dir))) 

total training cat image: 1000


In [ ]:
os.listdir(train_cats_dir)

# ImageDataGenerator class in keras

In [ ]:
#  ImageDataGenerator:

from keras.preprocessing.image import ImageDataGenerator

# step1: 
train_datagen=ImageDataGenerator(rescale=1./255) # 

validation_datagen=ImageDataGenerator(rescale=1./255)

# step2: 
train_generator= train_datagen.flow_from_directory(train_dir, 
                                                   target_size=(150,150), 
                                                   batch_size=20, 
                                                   class_mode='binary') # 

validation_generator= validation_datagen.flow_from_directory(validation_dir, 
                                                   target_size=(150,150),
                                                   batch_size=20,
                                                   class_mode='binary') #shuffel=True (defult), 

In [131]:
# calling  generator  ImageDataGenerator:

for data_batch, labels_batch in train_generator:
    print('data batch shape: ' , data_batch.shape) # (20, 150, 150, 3)
    print('labels batch shape: ' , labels_batch.shape) # (20,)
    break 

data batch shape:  (20, 150, 150, 3)
labels batch shape:  (20,)


# network architecture

In [33]:
from keras import layers
from keras import models
from tensorflow.keras import optimizers

model = models.Sequential()

model.add(layers.Conv2D(32, (3,3), activation='relu', input_shape=(150,150,3)))
model.add(layers.MaxPooling2D((2,2)))

model.add(layers.Conv2D(64, (3,3), activation='relu'))
model.add(layers.MaxPooling2D((2,2)))

model.add(layers.Conv2D(128,(3,3), activation='relu'))
model.add(layers.MaxPooling2D((2,2)))

model.add(layers.Conv2D(128,(3,3), activation='relu'))
model.add(layers.MaxPooling2D((2,2)))


model.add(layers.Flatten()) 
model.add(layers.Dropout(0.5))  
model.add(layers.Dense(512, activation='relu')) 
model.add(layers.Dense(1 , activation='sigmoid'))


model.compile(optimizer=optimizers.RMSprop(learning_rate=1e-4), 
                 loss='binary_crossentropy', 
                 metrics=['acc'])  # accuracy

In [35]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_24 (Conv2D)          (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d_24 (MaxPoolin  (None, 74, 74, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_25 (Conv2D)          (None, 72, 72, 64)        18496     
                                                                 
 max_pooling2d_25 (MaxPoolin  (None, 36, 36, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_26 (Conv2D)          (None, 34, 34, 128)       73856     
                                                                 
 max_pooling2d_26 (MaxPoolin  (None, 17, 17, 128)     

# training

In [ ]:
model_history=model.fit(train_generator, 
                    steps_per_epoch=100, #how many batches in each epoch, 
                    epochs=30, 
                    validation_data=validation_generator, 
                    validation_steps=50) #how many batches for validation

model.save('cats_vs_dogs_v1.h5') 

# data augmentation

In [129]:
train_datagen=ImageDataGenerator(rescale=1./255,
                          rotation_range=40, #  [-40 40] ,  rotation_range  (0 180)
                          width_shift_range=0.2, #  [-0.2 0.2]
                          height_shift_range=0.2,
                          shear_range=0.2, 
                          zoom_range=0.2, #  [1-0.2  1+0.2] 
                          horizontal_flip=True, 
                          fill_mode='nearest')


train_generator= train_datagen.flow_from_directory(train_dir, 
                                                   target_size=(150,150), 
                                                  batch_size=20, 
                                                  class_mode='binary') # 

validation_datagen=ImageDataGenerator(rescale=1./255) 


validation_generator= validation_datagen.flow_from_directory(validation_dir, 
                                                   target_size=(150,150),
                                                  batch_size=20,
                                                  class_mode='binary') 

In [ ]:
# training
model_history=model.fit(train_generator,
                  steps_per_epoch=100, 
                  epochs=30,
                  validation_data=validation_generator, 
                  validation_steps=50) 

In [ ]:
#overfit checking
history_dict=model_history.history

import matplotlib.pyplot as plt
loss_values_train=history_dict['loss'] #  loss  train
loss_values_val=history_dict['val_loss'] #  loss validation

epoches=range(1, len(loss_values_train)+1)

plt.plot(epoches,loss_values_train, 'bo',label='training')
plt.plot(epoches,loss_values_val,'r', label='validation')
plt.legend()
plt.show()